In [1]:
import csv
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing

In [2]:
def get_river_dataset(fname, pr_list=None, y_name='h_max'):
    pr_arr = []
    y_arr = []
    with open(fname, newline='') as f:
        reader = csv.DictReader(f, delimiter=';')
        for row in reader:
            pr_arr_row = []
            for col in pr_list:
                pr_arr_row.append(row[col])

            pr_arr.append(pr_arr_row)
            y_arr.append(row[y_name])
    X = np.asarray(pr_arr, dtype=np.float32)
    y = np.asarray(y_arr, dtype=np.float32)
    return X, y

#### Сумма, средний, высший, низший уровни

In [3]:
def get_sum(h_max):
    return np.sum(h_max)
    
def get_avg(h_max):
    return np.mean(h_max)
    
def get_max(h_max):
    return np.amax(h_max)
    
def get_min(h_max):
    return np.amin(h_max)

#### Среднеквадратическая погрешность прогноза S

In [4]:
def get_s(h_max, h_forecast=None):
    # Среднеквадратическая погрешность прогноза
    n = h_max.shape[0]
    sqr_diff = np.sum((h_max - np.around(h_forecast, 0)) ** 2) / (n - 1)
    std = sqr_diff ** 0.5
    return std    

#### Среднеквадратическое отклонение sigma

In [5]:
def get_sigma(h_max):
    # Среднеквадратическая погрешность климатическая.
    # Рассчитывается только по всей совокупности данных.
    n = h_max.shape[0]
    hmax_avg = (h_max - round(np.mean(h_max)))
    hmax_avg_sqr = hmax_avg ** 2
    sum_avg_sqr = np.sum(hmax_avg_sqr)
    std = (sum_avg_sqr / (n-1)) ** 0.5
    ##np.std(h_max, ddof=1)
    return std

In [6]:
def get_hmax_avg(h_max):
    # Среднее значение h_max.
    # Рассчитывается только по всей совокупности данных.
    return np.mean(h_max)

#### Допустимая погрешность прогноза delta_dop

In [7]:
def get_delta_dop(sigma):
    return 0.674 * sigma

#### Критерий эффективности метода прогнозирования климатический S/sigma

In [8]:
def get_criterion(s, sigma):
    return s / sigma

#### Климатическая обеспеченность Pk

In [9]:
def get_pk(h_max, h_max_avg, delta_dop):
    #avg_level = np.mean(h_max)
    diff = np.abs(h_max - h_max_avg) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Обеспеченность метода (оправдываемость) Pm

In [10]:
def get_pm(h_max, h_forecast, delta_dop):
    diff = np.abs(h_max - h_forecast) / delta_dop
    trusted_values = diff[diff <= 1.0]
    m = trusted_values.shape[0]
    n = h_max.shape[0]
    return m / n * 100.00

#### Корреляционное отношение

In [11]:
def get_correlation_ratio(criterion):
    c_1 = (1 - criterion ** 2)
    ro = c_1 ** 0.5 if c_1 > 0 else 0
    return ro

#### Вероятная ошибка прогноза S'

In [12]:
def get_forecast_error(s):
    return 0.674 * s

#### Ошибки климатического/природного прогноза для каждого года delta50

In [13]:
def get_delta50(h_max, delta_dop, h_max_avg=None, h_max_forecast=None):
    if h_max_forecast is None:
        # delta50 климатическая
        return (h_max - h_max_avg) / delta_dop
    else:
        # delta50 прогноза
        return (h_max - h_max_forecast) / delta_dop
  

#### Функция записи в csv файл

In [14]:
import csv
def write_dataset_csv(dataset, filename, fieldnames):
    with open(f'results/{filename}.csv', 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter=';', extrasaction='ignore')
        writer.writeheader()
        writer.writerows(dataset)


#### Функция разделения набора данных на тренировочный и тестовый

In [15]:
def train_test_split(X, y, n_test):
    X_train = X[:-n_test]
    y_train = y[:-n_test]
    X_test = X[-n_test:]
    y_test = y[-n_test:]
    return X_train, y_train, X_test, y_test

#### Функция получения датасетов

In [16]:
def get_datasets():
    datasets = {
        'Неман-Белица': 'Неман',
        'Неман-Гродно': 'Неман',
    }
    return datasets

#### Функция получения списка предикторов по названию датасета

In [17]:
def get_predictors(dataset_name):

    datasets = get_datasets()   
    
    predictors_lists = {
        'Неман': ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    }

    # predictors_lists = {
    #     'Неман': (
    #         ['s_2802', 's_max', 'h', 'x', 'x1', 'x2', 'x3', 'x4', 'xs'],
    #         ['s_max', 'h', 'x', 'x1', 'x3'],
    #         ['s_2802', 'h', 'x2', 'x3', 'xs'],
    #     )
    # }
    return predictors_lists[datasets[dataset_name]]
    

#### Функция обучения и оценки моделей

In [18]:
def compare_models(n_test=None, norms=False):
    from sklearn.linear_model import LinearRegression
    from sklearn.linear_model import Ridge
    from sklearn.linear_model import Lasso
    from sklearn.linear_model import ElasticNet, ElasticNetCV
    from sklearn.linear_model import Lars, LarsCV
    from sklearn.linear_model import LassoLars

    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF
    
    
    ds_dir = 'data' # В константы
    
    names = [
        
        #'Ridge',
        #'Lasso',
        #'ElasticNet',
        #'ElasticNetCV',
        #'LassoLars',
        #'Lars1',
        #'Lars2',
        #'Lars3',
        #'Lars4',
        #'Lars5',
        #'Lars6',
        #'Lars7',
        #'Lars8',
        
        'LarsCV',
        'LinearRegression',
        
        #'GaussianProcessRegressor',
        
    ]

    regressors = [
        
        #Ridge(alpha=10),
        #Lasso(alpha=100.0),
        #ElasticNet(alpha=2.0, l1_ratio=0.1),
        #ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], eps=0.01, n_alphas=1000),
        #LassoLars(alpha=1.0),
        #Lars(n_nonzero_coefs=1),
        #Lars(n_nonzero_coefs=2),
        #Lars(n_nonzero_coefs=3),
        #Lars(n_nonzero_coefs=4),
        #Lars(n_nonzero_coefs=5),
        #Lars(n_nonzero_coefs=6),
        #Lars(n_nonzero_coefs=7),
        #Lars(n_nonzero_coefs=8),
        
        LarsCV(max_iter=5000, max_n_alphas=10000, cv=5),
        LinearRegression(),
        #GaussianProcessRegressor(kernel=RBF(length_scale=1.1) + WhiteKernel() + DotProduct(), random_state=0)
    ]

    datasets = get_datasets()
    
    norms = {
        
    }

    fieldnames = ['Predictors', 'Equations', 'Method', 'Criterion', 'Correlation', 'Pm']

    # datasets_result = {
    #     "hydropost_0": [
    #         { model_row }
    #         { model_row }
    #     ],
    #     ...,
    #     "hydropost_n": [
    #         { model_row }
    #         { model_row }
    #     ],
    # }
    
    
    # Итерация по датасетам
    datasets_result = dict()
    for ds in datasets:
        result_list = []
        #datasets_result[ds] = []
        # one_dataset_row = dict()
        
        pr_list = get_predictors(ds)
        
        X, y = get_river_dataset(f'{ds_dir}/{ds}.csv', pr_list=pr_list)

        if n_test is not None and n_test != 0:
            X_train, y_train, X_test, y_test = train_test_split(X, y, n_test)
        else:
            X_train = X[:]
            y_train = y[:]
            X_test = X_train
            y_test = y_train
            
        # Итерация по моделям регрессии
        # models_list = []
        for name, regr in zip(names, regressors):
            one_model_row = dict()
                
            regr.fit(X_train, y_train)
            y_predicted = regr.predict(X_test)
            
            # Коэффициенты уравнения (если есть)
            try:
                predictors_coef = {f: c for f, c in zip(pr_list, regr.coef_) if c != 0.0}
                predictors = ", ".join(predictors_coef.keys())
                equation = ' '.join(str(round(c, 2))+'*'+f for f, c in predictors_coef.items())
                equation = equation.replace(" -", "-")
                equation = equation.replace(" ", " + ")
                equation = equation.replace("-", " - ")
    
                one_model_row['Predictors'] = predictors
                one_model_row['Equations'] = equation
            except Exception:
                one_model_row['Predictors'] = ""
                one_model_row['Equations'] = ""

            # Название метода
            one_model_row['Method'] = name

            # Расчет показателей качества по методике
            
            # Среднее значение максимального уровня по всей выборке
            h_max_avg = round(get_hmax_avg(y), 0)
            one_model_row['H_avg'] = h_max_avg
            
            # Среднеквадратическое отклонение
            sigma = get_sigma(y)
            one_model_row['Sigma'] = sigma

            # Допустимая погрешность прогноза
            delta_dop = get_delta_dop(sigma)
            one_model_row['Delta_dop'] = delta_dop

            # Обеспеченность климатическая Pk
            pk = get_pk(y_test, h_max_avg, delta_dop)
            one_model_row['Pk'] = round(pk, 1)

            # Обеспеченность метода (оправдываемость) Pm
            pm = get_pm(y_test, y_predicted, delta_dop)
            one_model_row['Pm'] = round(pm, 1)

            # Среднеквадратическая погрешность прогноза
            s_forecast = get_s(y_test, y_predicted)
            one_model_row['S'] = s_forecast
            
            # Критерий эффективности метода прогнозирования климатический S/sigma
            criterion_forecast = get_criterion(s_forecast, sigma)
            one_model_row['Criterion'] = criterion_forecast

            # Критерий эффективности метода прогнозирования климатический S/sigma в квадрате
            criterion_sqr = get_criterion(s_forecast, sigma) ** 2.0
            one_model_row['Criterion_sqr'] = criterion_sqr
            
            # Корреляционное отношение ro
            correlation_forecast = get_correlation_ratio(criterion_forecast)
            one_model_row['Correlation'] = correlation_forecast
                        
            # Model
            one_model_row['Model'] = regr

            # models_list.append(one_model_row)
            result_list.append(one_model_row)

        # Сортировка результатов по каждому датасету
        result_list.sort(key=lambda row: (row['Criterion'], -row['Correlation'], -row['Pm']))

        datasets_result[ds] = result_list
        #datasets_result.append(one_dataset_row)

        # Запись в .csv файл
        write_dataset_csv(result_list, ds, fieldnames)

        verify_forecast(ds, result_list[0]['Model'], n_test=n_test)    

    

    return datasets_result

#### Функция формирования проверочных прогнозов

In [19]:
def verify_forecast(dataset_name, model, n_test=None):

    ds_dir = 'data' # В константы

    pr_list = get_predictors(dataset_name)
    pr_list = ['year'] + pr_list
    
    # fieldnames = [
    #     'Год',
    #     'Hmax фактический', 'Hф-Hср', '(Hф-Hср)^2', 'Погрешность климатических прогнозов в долях от допустимой погрешности',
    #     'Hmax прогнозный', 'Hф-Hп', '(Hф-Hп)^2', 'Погрешность проверочных прогнозов в долях от допустимой погрешности',
    # ]
    fieldnames = [
        'Year',
        'Hmax fact', 'Hf-Havg', '(Hf-Havg)^2', 'Error climate',
        'Hmax forecast', 'Hf-Hfor', '(Hf-Hfor)^2', 'Error forecast',
    ]

    X, y = get_river_dataset(f'{ds_dir}/{dataset_name}.csv', pr_list=pr_list, y_name='h_max')

    if n_test is not None and n_test != 0:
        _, _, X_test, y_test = train_test_split(X, y, n_test)
    else:
        X_test = X
        y_test = y

    # Выделение первой колонки (года) из набора предикторов
    years = np.asarray(X_test[:, 0], dtype=np.float32)
    X_test = np.asarray(X_test[:, 1:], dtype=np.float32)
    
    # Forecast
    h_max_forecast = np.around(model.predict(X_test))
    
    # Hсредний
    h_max_avg = np.around(np.mean(y))

    # H - Hсредний
    diff_fact = y_test - h_max_avg

    # (H - Hсредний) в квадрате
    diff_fact_sqr = diff_fact ** 2

    # Погрешность климатических прогнозов в долях от допустимой погрешности
    delta_dop = get_delta_dop(get_sigma(y))
    error_climate = np.around(get_delta50(y_test, delta_dop, h_max_avg=h_max_avg), decimals=2)

    # H - Hпрогнозный
    diff_forecast = y_test - h_max_forecast

    # (H - Hпрогнозный) в квадрате
    diff_forecast_sqr = diff_forecast ** 2       

    # Погрешность проверочных прогнозов в долях от допустимой погрешности
    error_forecast = np.around(get_delta50(y_test, delta_dop, h_max_forecast=h_max_forecast), decimals=2)

    # Конкатенация массивов
    att_tuple = (years, y_test, diff_fact, diff_fact_sqr, error_climate, h_max_forecast, diff_forecast, diff_forecast_sqr, error_forecast)
    arr = np.column_stack(att_tuple)
    arr = arr.tolist()

    # Обеспеченность метода (оправдываемость) Pm
    pm = get_pm(y_test, h_max_forecast, delta_dop)
    
    # Запись проверочного прогноза в csv файл
    with open(f'results/{dataset_name}-проверочный.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile, delimiter=';')
        writer.writerow(fieldnames)
        writer.writerows(arr)
        
      
        
        

In [20]:
result = compare_models(n_test=0)
for key in result:
    print(key)
    for r in result[key]:
        print(
            f"{r['Method']:>16},\n" 
            f"H_avg={r['H_avg']},\n" 
            f"Sigma={r['Sigma']},\n" 
            f"Delta_dop={r['Delta_dop']},\n" 
            f"Pk={r['Pk']},\n"
            f"Pm={r['Pm']},\n"
            f"S={r['S']},\n"
            f"Criterion={r['Criterion']},\n" 
            f"Criterion^2={r['Criterion_sqr']},\n"
            f"Correlation={r['Correlation']},\n"            
            f"Model={r['Model']}\n\n"
        )


Неман-Белица
LinearRegression,
H_avg=290.0,
Sigma=72.81693179168947,
Delta_dop=49.07861202759871,
Pk=37.8,
Pm=75.7,
S=37.23498951852201,
Criterion=0.5113507065230618,
Criterion^2=0.26147954506163446,
Correlation=0.8593721283229783,
Model=LinearRegression()


          LarsCV,
H_avg=290.0,
Sigma=72.81693179168947,
Delta_dop=49.07861202759871,
Pk=37.8,
Pm=78.4,
S=41.43602833820399,
Criterion=0.5690438654671941,
Criterion^2=0.3238109208258461,
Correlation=0.8223071683830525,
Model=LarsCV(cv=5, max_iter=5000, max_n_alphas=10000)


Неман-Гродно
LinearRegression,
H_avg=280.0,
Sigma=96.45206063117574,
Delta_dop=65.00868886541245,
Pk=51.4,
Pm=85.7,
S=50.236206769502765,
Criterion=0.5208411975935033,
Criterion^2=0.2712755531106348,
Correlation=0.8536535871706773,
Model=LinearRegression()


          LarsCV,
H_avg=280.0,
Sigma=96.45206063117574,
Delta_dop=65.00868886541245,
Pk=51.4,
Pm=80.0,
S=58.19920153080736,
Criterion=0.6034002918129041,
Criterion^2=0.36409191215989783,
Correlation=0.7974384